In [17]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load the .env file

True

In [18]:
from llama_index.llms.groq import Groq
from llama_index.core import VectorStoreIndex, load_index_from_storage, StorageContext

llm = Groq(model="llama3-8b-8192", api_key=os.getenv("GROQ_API_KEY"))

In [19]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from chromadb import PersistentClient

# Load the embedding model explicitly
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

persist_dir = "./chroma_db1"
client = PersistentClient(path=persist_dir)
collection = client.get_or_create_collection("rag-collection")
vector_store = ChromaVectorStore(chroma_collection=collection, persist_dir=persist_dir)

# Use the same embedding model while loading index
storage_context = StorageContext.from_defaults(
    persist_dir="./index",
    vector_store=vector_store
)

index = load_index_from_storage(storage_context, embed_model=embed_model)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index\index_store.json.


In [20]:
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("What is the admission process at PU?")
print(response)

The admission process at PU involves submitting an admission form, with an additional fee for each additional course applied for. Candidates intending to apply for more than 8 courses must submit another admission form. There are different dates for normal admission, late admission with permission of the chairperson, and late admission with permission of the V.C. Ongoing classes will start from a specified date, and candidates whose result of the lower semester class has not been declared must seek provisional admission.


In [21]:
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("What is the admission criteria?")
print(response)

For admission to the course, a candidate's merit shall be determined on the basis of their score, which is computed as follows: (a) percentage of marks in the qualifying examination, (b) add score of 4 for each of the subjects passed from the subjects referred in 2(1)(a), not exceeding 16 in total, (c) 2 examinations with at least two of the subjects mentioned in (a) securing at least 50 marks in aggregate, (d) 2 examinations who do not cover in (a) and (b) securing at least 55 marks, or (e) any other examination recognized by the university as equivalent to (a), (b), or (c) with requisite percentage of marks.


In [43]:
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("Tell me about the fees structure")
print(response)

The tuition fee for the University Institute of Engineering and Technology is $3600 for the first year, with a total fee of $4000. The fee for the second and third years is the same, at $4000.


In [44]:
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("Admission in Law?")
print(response)

No, applicants who have obtained 10+2 through open universities system directly without having any basic qualification for prosecuting such studies are not eligible for admission in the law course.


In [22]:
from sentence_transformers import CrossEncoder

# Load a cross-encoder model for re-ranking
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def rerank(query, docs):
    pairs = [[query, doc] for doc in docs]
    scores = reranker.predict(pairs)
    sorted_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
    return sorted_docs

In [54]:
query = "How to get admission in UIET?"

# Retrieve top-10 docs from vector store
retriever = index.as_retriever(similarity_top_k=10)
nodes = retriever.retrieve(query)
doc_texts = [node.get_content() for node in nodes]

# Re-rank and pick top 3
top_docs = rerank(query, doc_texts)[:3]

# Combine top docs into a single context string
context = "\n\n---\n\n".join(top_docs)

# Format the prompt
prompt = f"""
PU stands for Panjab University, Chandigarh and you are PU-Assistant, a helpful and professional virtual guide for students of Panjab University and the students who wish to take admission here.

Use only the context provided below to answer the student's question. 
Follow these instructions strictly:

1. If the answer involves steps, forms, rules, or eligibility — present the information in **clear bullet points**.
2. If the context includes a **URL or downloadable form**, mention it politely at the end of the answer.
3. If the answer is straightforward (like a definition or a one-liner), give a **concise sentence**.
4. If any **URL or website link is relevant to the answer**, be sure to **provide it clearly** in your response.
5. If the answer is not found in the context, reply: "**Sorry, I couldn't find that information. You may contact the administration.**" or "**Sorry, I couldn't help you with that. You may visit the website.**" based on what type of question was asked.
6. Maintain a **polite and formal tone**, like a university helpdesk.
7. Do not use information outside the provided context.
8. At the end of your answer, list three possible follow-up questions as **Know more about:**, each written in **no more than 5 words**.

Context:
{context}

Question: {query}

Answer:
"""


response = llm.complete(prompt)
print(response.text)

To get admission in University Institute of Engineering Technology (UIET), Panjab University, Chandigarh, you can follow these steps:

* Check the vacant seats available in the desired course.
* Fill out the online admission form and submit it along with the required documents and fee.
* For reserved category of sports, take a printout of the online admission form and submit self-attested photocopies of the relevant certificates and undertakings to the Office of the Campus Sports, PU, Chandigarh.
* For foreign national students, obtain an eligibility equivalency certificate from the Deputy Registrar (General), Panjab University, Chandigarh, or academic certificates of passing the qualifying or equivalent examination approved by the Association of Indian Universities (AIU), Common Wealth Universities, or International Association of Universities (IAU) as equivalent to the corresponding Indian degrees.
* Submit the eligibility certificate along with the required documents and fee.
* For 